In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import uuid
from sortedcontainers import SortedList

points = np.random.randn(100, 2)
plt.scatter(points[:, 0], points[:, 1])
data = [(str(uuid.uuid4()), p) for p in points]

In [ ]:
# data should look like this
# {uid of the point: corrdinate in d-dimension}

class TKM:
    
    def __init__(self, k=3, tol=1e-4, max_iter=300):
        self.k = k
        self.tol = tol
        self.max_iter = max_iter
    
    def compute_threshold(self, data, point):
        pass
    
    def fit(self, data):
        self.d = {}
        self.d_prime = {}
        self.centers = {}
        self.classifications = {}
        self.m = {}
        self.SUM = {}
        self.sigma = 0
        
        # Initialize centroids (or centers)
#         np.random.shuffle(data)
        for i in range(self.k):
            self.centers[i] = data[i][1]
            self.m[data[i][0]] = i
            self.classifications[i] = []
            
        for p in data:
            distances = [np.linalg.norm(p[1]-self.centers[c]) for c in self.centers]
            classification = np.argmin(distances)
            self.classifications[classification].append(p)
            self.m[p[0]] = classification
            
        # Initialize d and d_prime
        for p in data:
            self.d[p[0]] = np.linalg.norm(p[1]-self.centers[self.m[p[0]]])
            cur_min = None
            for c in self.centers:
                if c==self.m[p[0]]:
                    continue
                if cur_min==None:
                    cur_min = np.linalg.norm(p[1]-self.centers[c])
                else:
                    cur_min = min(cur_min, np.linalg.norm(p[1]-self.centers[c]))
            self.d_prime[p[0]] = cur_min
            
        # Initialize P_active
        ## H is a binary search tree of tuple (d-d', uid)
        self.H = SortedList([(self.d_prime[p[0]]-self.d[p[0]], p[1], p[0]) for p in data], key=lambda x:x[0])
        self.P_active = list(self.H.islice(0, self.H.bisect_left((0, None, None))))
        
        # Initialize SUM
        for c in self.centers:
            self.SUM[c] = np.vstack([k[1] for k in self.classifications[c]]).sum(axis=0)   
            
        # Main loop
        ok = False
        for i in range(self.max_iter):
            ok = True
            for p in self.P_active:
                distances = [np.linalg.norm(p[1]-self.centers[c]) for c in self.centers]
                nearest = np.argmin(distances)
                if nearest != self.m[p[2]]:
                    self.SUM[self.m[p[2]]] = np.subtract(self.SUM[self.m[p[2]]], p[1])
                    self.SUM[nearest] = np.add(self.SUM[nearest], p[1])
                    self.m[p[2]] = nearest
            for c in self.centers:
                m_g = self.SUM[c]/len(self.classifications[c])
                self.sigma = max(self.sigma, np.linalg.norm(self.centers[c]-m_g))
                if np.sum((self.centers[c]-m_g)/self.centers[c] * 100.0) > self.tol:
                    ok = False
                self.centers[c] = m_g
            if ok:
                break
            self.P_active = list(self.H.islice(0, self.H.bisect_left((self.sigma, None, None))))

    def UB_Y(p, m_p, theta_p, M, delta):
        d1 = np.linalg.norm(p-m_p)
        cur_min = None
        for m in M:
            if np.array_equal(m, m_p):
                continue
            if cur_min==None:
                cur_min = np.linalg.norm(p, m)
            else:
                cur_min = min(cur_min, np.linalg.norm(p, m))
        return np.square(d1+delta+theta_p) - np.square(max(0, cur_min-delta-theta_p))

    # P is a list of tuple (uuid, vector)
    # m is a map from uuid of the point to the index of the point's corresponding center 
    # (e.g. m[p_1]=2 means M[2] is m[p_1]'s clustering center)
    # classifications is a map from the index of the center to the list of points belonging to this center
    def compute_threshold(P, M, m, classifications, delta_1, delta_2):
        delta = delta_1 + delta_2
        Theta = {}
        P_move = []
        for p in P:
            Theta[p[0]] = delta_1
        for p in P:
            if UB_Y(p[1], M[m[p[0]]], Theta[p[0]], M, delta) < 0:
                P_move.append(p)
        ok = False
        while not ok:
            ok = True
            d_p = {}
            d_p_prime = {}
            for p in P_move:
                d_p[p[0]] = np.linalg.norm(p[1]-M[m[p[0]]])
                d_p_prime[p[0]] = None
                for i in range(len(M)):
                    if i == m[p[0]]:
                        continue
                    if d_p_prime[p[0]] == None:
                        d_p_prime[p[0]] = np.linalg.norm(p[1]-M[i])
                    else:
                        d_p_prime[p[0]] = min(np.linalg.norm(p[1]-M[i]), d_p_prime[p[0]])
            for p in P_move:
                # Decrease theta_p so that it satisfies theta_p <= minm∈M\{mp}dist(p,m)−Δ
                Theta[p[0]] = d_p_prime[p[0]] - delta
                # Let theta_p_aster be results of evaluating Equation 4.11
                sum_1 = np.sum([np.sqrt(d_p[q[0]]+d_p_prime[q[0]]) for q in P_move])
                sum_2 = np.sum([np.square(d_p[q[0]]+delta)-np.square(d_p_prime[q[0]]-delta) for q in P_move])
                min_cardi = np.min([len(x) for x in classifications.values()])
                theta_p_aster = np.sqrt(d_p[p[0]]-d_p_prime[p[0]])/(2*(d_p[p[0]]+d_p_prime[p[0]])*sum_1)*(min_cardi*np.square(delta_2)-sum_2)
                if theta_p_aster < Theta[p[0]]:
                    Theta[p[0]] = theta_p_aster
            for p in P_move:
                if UB_Y(p[1], M[m[p[0]]], Theta[p[0]], M, delta) <= 0:
                    P_move.remove(p)
                    ok = False